In [71]:
from langchain.chat_models import ChatOpenAI
from langchain import HuggingFaceHub
from langchain.prompts import (
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser

In [72]:
from dotenv import dotenv_values
dotenv_values = dotenv_values()
hf_key = dotenv_values['HUGGINGFACEHUB_API_TOKEN']
openai_key = dotenv_values['OPENAI_API_KEY']

In [73]:
model = ChatOpenAI(temperature=1, openai_api_key=openai_key)
model.model_name


'gpt-3.5-turbo'

In [74]:
template = """The user will provide the initial part of the function (function name, parameters with types, return type and a comment describing the purpose of the function, with some optional example.
You are a coding assistent that writes some python code to complete the user's function. Be sure that the code is syntactically correct and that it returns the correct type. The lenght of the code should be less than {max_lines} lines.

Return only python code in Markdown format, e.g.:

```python
....
```"""

In [75]:
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])


In [76]:
def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

In [77]:
chain = prompt | model | StrOutputParser() | _sanitize_output

In [78]:
input = '''def detect_xss(http_get_request: str)->bool: 
""" Check if in the given http_get_request there is an XSS exploit, considering also the possible evansions that an attacker can perform.""" 
'''

In [79]:
prompt

ChatPromptTemplate(input_variables=['input', 'max_lines'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['max_lines'], template="The user will provide the initial part of the function (function name, parameters with types, return type and a comment describing the purpose of the function, with some optional example.\nYou are a coding assistent that writes some python code to complete the user's function. Be sure that the code is syntactically correct and that it returns the correct type. The lenght of the code should be less than {max_lines} lines.\n\nReturn only python code in Markdown format, e.g.:\n\n```python\n....\n```")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [80]:
response = chain.invoke({"max_lines":50,"input": input})

In [81]:
print(response)


import re

def detect_xss(http_get_request: str) -> bool: 
    """Check if in the given http_get_request there is an XSS exploit, considering also the possible evasions that an attacker can perform.""" 
    # Define the patterns of common XSS attack vectors
    patterns = [
        r"<[^>]+>",
        r"\"[^<]+\"",
        r"\'[^<]+\'",
        r"\([^<]+\)",
        r"[^<]+\([^<]+\)",
        r"[^<]+\(.*\)",
        r"javascript:",
        r"on\w*=",
        r"src\s*=.*\.(jpg|jpeg|gif|png|bmp|tif)",
    ]

    # Check if any pattern matches the http_get_request
    for pattern in patterns:
        if re.search(pattern, http_get_request, re.IGNORECASE):
            return True
    
    return False

